In [ ]:
import mesh_utils
import numpy as np
import importlib
importlib.reload(mesh_utils)

## For converting the raw .vtp and .vtu files to openCARP format:

In [ ]:
data_path = "../data/instance_001/"
vtp_path = data_path + "instance_001.vtp"
vtu_path = data_path + "instance_001.vtu"
output_prefix = "instance_001"

mesh_utils.vtk_to_opencarp(vtp_path, vtu_path, output_prefix, data_path)

Reading surface mesh: ../data/instance_001/instance_001.vtp
Reading volume mesh: ../data/instance_001/instance_001.vtu
Converting to openCARP format. Output directory: ../data/instance_001/
Extracted tv shape: (478820,)
tv range: 0.0 to 1.0
Extracted tm shape: (478820,)
tm range: 0.0 to 1.0
Extracted rtSin shape: (478820,)
rtSin range: -1.0 to 1.0
Extracted rtCos shape: (478820,)
rtCos range: -1.0 to 0.9999998807907104
Extracted rt shape: (478820,)
rt range: 1.2781016494045616e-06 to 0.999995768070221
Extracted ab shape: (478820,)
ab range: 0.0 to 1.0
Saved UVC data to ../data/instance_001/instance_001_UVC.csv
Created ../data/instance_001/instance_001.pts with 478820 points
Created instance_001.elem with tetrahedral elements
Created instance_001.surf with surface triangles
Reading points from ../data/instance_001/instance_001.pts
Loaded 478820 points with shape (478820, 3)
Scaled points by factor of 1000
Saved scaled points back to ../data/instance_001/instance_001.pts
Conversion compl

## Read in the points, surfaces, tetrahedra, and UVCs

In [2]:
prefix = "instance_001_lowres"
data_path = f"/work/submit/mcgreivy/full-heart-simulation/data/{prefix}"

reader = mesh_utils.OpenCARPMeshReader(data_path, prefix)
points, tetrahedra, tetrahedra_regions, triangles, triangle_regions, uvc_data = reader.read_all()

Reading points from /work/submit/mcgreivy/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.pts
Loaded 210411 points with shape (210411, 3)
Reading tetrahedra from /work/submit/mcgreivy/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.elem
Loaded 1040683 tetrahedra with shape (1040683, 4)
Reading triangles from /work/submit/mcgreivy/full-heart-simulation/data/instance_001_lowres/instance_001_lowres.surf
Loaded 158252 triangles with shape (158252, 3)
Reading UVC data from /work/submit/mcgreivy/full-heart-simulation/data/instance_001_lowres/instance_001_lowres_UVC.csv
Loaded UVC data with shape (210411, 6)
UVC columns: ['tv', 'tm', 'rtSin', 'rtCos', 'rt', 'ab']


## Use the UVCs to define a coordinate system and compute the fiber and sheet directions

In [7]:
phi_transmural = np.array(uvc_data['tm'])
phi_longitudinal = np.array(uvc_data['ab'])
phi_circumferential = np.array(uvc_data["rt"])

In [6]:
TransmuralField, LongitudinalField, CircumferentialField = mesh_utils.compute_normed_gradients(points, -phi_transmural, -phi_longitudinal, phi_circumferential)

NameError: name 'phi_transmural' is not defined

In [17]:
mesh_utils.visualize_vector_fields(
    points, triangles, triangle_regions,
    TransmuralField, LongitudinalField, CircumferentialField, subsample_factor=20, glyph_scale=5000
)

/work/submit/mcgreivy/miniconda3/envs/openCARP/lib/python3.10/site-packages/pyvista/plotting/plotter.py:159: UserWarning: 
This system does not appear to be running an xserver.
PyVista will likely segfault when rendering.

Try starting a virtual frame buffer with xvfb, or using
  ``pyvista.start_xvfb()``

  warnings.warn(
/work/submit/mcgreivy/miniconda3/envs/openCARP/lib/python3.10/site-packages/pyvista/jupyter/notebook.py:37: UserWarning: Failed to use notebook backend: 

No module named 'trame'

Falling back to a static output.
  warnings.warn(
ERROR:root:bad X server connection. DISPLAY=
2025-03-08 11:43:47.293 ( 763.834s) [    7F3370EE9740]vtkXOpenGLRenderWindow.:456    ERR| vtkXOpenGLRenderWindow (0x48f6ad0): bad X server connection. DISPLAY=


: 

In [ ]:
fiber_dirs, sheet_dirs, sheet_normal_dirs = mesh_utils.compute_fiber_sheet_directions(TransmuralField, LongitudinalField, CircumferentialField, phi_transmural, 
                                                                                      endo_fiber_angle=60.0, epi_fiber_angle=-60, endo_sheet_angle=-65, epi_sheet_angle=25
)

mesh_utils.visualize_fibers(
    points, triangles, triangle_regions,
    fiber_dirs, subsample_factor=10, glyph_scale=5000
)

In [ ]:
mesh_utils.write_lon_file(
    f"{data_path}/{prefix}.lon", 
    fiber_dirs, 
    sheet_dirs, 
    sheet_normal_dirs,
    tetrahedra
)

Processing 2555157 elements for .lon file...


Precomputing element directions: 100%|██████████| 2555157/2555157 [00:44<00:00, 58003.73it/s]


Verifying orthogonality of element directions...
Maximum dot products: fiber·sheet=0.999987, fiber·normal=0.999816, sheet·normal=0.999924
Writing to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/instance_001.lon...


Writing .lon file: 100%|██████████| 256/256 [00:05<00:00, 44.94it/s]

Successfully wrote fiber orientations to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/instance_001.lon


## Tag the fast conducting endocardium

In [ ]:
mesh_utils.tag_fast_conducting_endocardium(
    points, 
    tetrahedra, 
    triangles, 
    triangle_regions, 
    phi_longitudinal, 
    f"{data_path}/{prefix}.elem",
    long_min = 0.2,
    long_max = 0.85
)

Identifying points with phi_longitudinal between 0.2 and 0.85...
Found 356093 points with valid longitudinal coordinate
Identifying endocardial points...
Found 52674 points on the endocardium
Found 38338 points that satisfy both criteria
Tagging tetrahedra...


100%|██████████| 256/256 [00:01<00:00, 136.60it/s]


Tagged 259191 tetrahedra (10.14%) as fast conducting endocardium
Writing modified element file to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/instance_001.elem...


100%|██████████| 2555157/2555157 [00:02<00:00, 875736.89it/s]

Successfully wrote modified element file to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/instance_001.elem


## Tag the fascicular sites and output to a .vtx file

In [63]:
import numpy as np

def tag_fascicular_sites(points, phi_transmural, phi_longitudinal, phi_circumferential, 
                          triangles, triangle_regions):
    """
    Tag fascicular sites according to the Durrer-based model.
    
    Parameters:
    ----------
    points : numpy.ndarray
        Vertices of the mesh
    phi_transmural : numpy.ndarray
        Transmural coordinates (0 on epicardium, 1 on endocardium)
    phi_longitudinal : numpy.ndarray
        Longitudinal coordinates (1 on base, 0 on apex)
    phi_circumferential : numpy.ndarray
        Circumferential coordinates (wraps around from 0-1)
    triangles : numpy.ndarray
        Triangle indices
    triangle_regions : numpy.ndarray
        Region labels for each triangle (3 for LV endocardium, 4 for RV endocardium)
    
    Returns:
    -------
    is_fascicular_site : numpy.ndarray
        Boolean array indicating whether each point is a fascicular site
    fascicular_site_tag : numpy.ndarray
        Integer array with tags: 0 (not a site), 1 (LV anterior), 2 (LV posterior), 
        3 (LV/RV septal), 4 (RV moderator band)
    """
    # Constants based on the criteria
    transmural_depth = 1.0  # Just pick points directly on the endocardium
    disk_thickness = 0.2  # 0.5% of the ventricular wall
    
    # Initialize arrays
    num_points = len(points)
    is_fascicular_site = np.zeros(num_points, dtype=bool)
    fascicular_site_tag = np.zeros(num_points, dtype=int)
    
    # Define disk radius for endocardial extent
    disk_radius = 0.025  # Adjust this parameter as needed
    
    # Get indices of LV and RV endocardial points
    LV_points = np.array(list(set(triangles[triangle_regions == 3].flatten())))
    RV_points = np.array(list(set(triangles[triangle_regions == 4].flatten())))
    
    # Calculate centers for LV and RV
    LV_center = points[LV_points].mean(axis=0)
    RV_center = points[RV_points].mean(axis=0)
    
    # Calculate septal normal vector (from LV to RV)
    septal_normal = RV_center - LV_center
    septal_normal = septal_normal / np.linalg.norm(septal_normal)
    
    # Helper function to determine if a point is closer to LV or RV
    def is_closer_to_LV(point):
        mid_point = (LV_center + RV_center) / 2
        distance = np.dot(point - mid_point, septal_normal)
        return distance < 0  # Negative means closer to LV, positive means closer to RV
    
    # 1. LV anterior site
    lv_ant_circum = 0.25  # Around 1/4 of the way through circumferential coordinate
    lv_ant_longit = 0.5   # Middle of the LV along longitudinal axis
    
    lv_ant_mask = np.logical_and(
        np.abs(phi_transmural - transmural_depth) < disk_thickness/2,
        np.logical_and(
            (phi_circumferential - lv_ant_circum)**2 + (phi_longitudinal - lv_ant_longit)**2 < disk_radius**2,
            np.array([is_closer_to_LV(p) for p in points])
        )
    )
    
    is_fascicular_site = np.logical_or(is_fascicular_site, lv_ant_mask)
    fascicular_site_tag[lv_ant_mask] = 1
    
    # 2. LV posterior site
    lv_post_circum = 0.6  # Around 3/4 of the way through circumferential coordinate
    lv_post_longit = 0.5   # Middle of the LV along longitudinal axis
    
    lv_post_mask = np.logical_and(
        np.abs(phi_transmural - transmural_depth) < disk_thickness/2,
        np.logical_and(
            (phi_circumferential - lv_post_circum)**2 + (phi_longitudinal - lv_post_longit)**2 < disk_radius**2,
            np.array([is_closer_to_LV(p) for p in points])
        )
    )
    
    is_fascicular_site = np.logical_or(is_fascicular_site, lv_post_mask)
    fascicular_site_tag[lv_post_mask] = 2
    
    # 3. Septal sites (LV and RV)
    septal_anterior_circum = 0.95  # Anterior septal wall
    septal_longit = 0.5          # Middle along longitudinal axis
    
    septal_mask = np.logical_and(
        np.abs(phi_transmural - transmural_depth) < disk_thickness/2,
        (phi_circumferential - septal_anterior_circum)**2 + (phi_longitudinal - septal_longit)**2 < disk_radius**2
    )
    
    is_fascicular_site = np.logical_or(is_fascicular_site, septal_mask)
    fascicular_site_tag[septal_mask] = 3
    
    # 4. RV moderator band
    rv_mod_circum = 0.4      # Middle of RV free wall
    rv_mod_longit = 0.5      # Middle along longitudinal axis
    
    rv_mod_mask = np.logical_and(
        np.abs(phi_transmural - transmural_depth) < disk_thickness/2,
        np.logical_and(
            (phi_circumferential - rv_mod_circum)**2 + (phi_longitudinal - rv_mod_longit)**2 < disk_radius**2,
            np.logical_not(np.array([is_closer_to_LV(p) for p in points]))
        )
    )
    
    is_fascicular_site = np.logical_or(is_fascicular_site, rv_mod_mask)
    fascicular_site_tag[rv_mod_mask] = 4
    
    return is_fascicular_site, fascicular_site_tag


is_fascicular_site, fascicular_site_tag = tag_fascicular_sites(points, phi_transmural, phi_longitudinal, phi_circumferential, triangles, triangle_regions)

mesh_utils.visualize_fascicular_sites(
    points, triangles, triangle_regions,
    is_fascicular_site, fascicular_site_tag, 
    sphere_scale=1000,
)

immediate_stim = np.logical_and(is_fascicular_site, np.logical_or(fascicular_site_tag == 1, np.logical_or(fascicular_site_tag == 2, fascicular_site_tag == 3)))
mesh_utils.save_fascicular_sites_to_vtx(immediate_stim, output_filename=f"{data_path}/immediate_stim.vtx")
delayed_stim = np.logical_and(is_fascicular_site, fascicular_site_tag == 4)
mesh_utils.save_fascicular_sites_to_vtx(delayed_stim, output_filename=f"{data_path}/delayed_stim.vtx")

Original points: 478820
Original triangles: 199902
Number of fascicular sites: 307
Endocardium triangles: 104504
Displaying all 307 fascicular sites


/Users/jamesmcgreivy/miniconda3/envs/openCARP/lib/python3.10/site-packages/pyvista/core/filters/data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:61771/index.html?ui=P_0x33df4bb20_21&reconnect=auto" class="pyvist…

Saving 221 fascicular site indices to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/immediate_stim.vtx
Successfully saved fascicular sites to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/immediate_stim.vtx
Saving 86 fascicular site indices to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/delayed_stim.vtx
Successfully saved fascicular sites to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/delayed_stim.vtx


'/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/delayed_stim.vtx'

##  Extra utility function -- for visualizing the scalar fields

In [9]:
mesh_utils.visualize_phi(points, phi_transmural, subsample_factor=1, point_size=5)

: 